In [15]:
import pandas as pd

# Load the dataset
try:
    df = pd.read_csv('/content/iphone.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('/content/iphone.csv', encoding='latin-1')

# Display the first few rows and the columns
display(df.head())
display(df.info())

,productAsin,country,date,isVerified,ratingScore,reviewTitle,reviewDescription,reviewUrl,reviewedIn,variant,variantAsin
0,B09G9BL5CP,India,11-08-2024,True,4,No charger,"Every thing is good about iPhones, there's not...",https://www.amazon.in/gp/customer-reviews/R345...,Reviewed in India on 11 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
1,B09G9BL5CP,India,16-08-2024,True,5,iPhone 13 256GB,"It look so fabulous, I am android user switche...",https://www.amazon.in/gp/customer-reviews/R2HJ...,Reviewed in India on 16 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
2,B09G9BL5CP,India,14-05-2024,True,4,Flip camera option nill,I tried to flip camera while recording but no ...,https://www.amazon.in/gp/customer-reviews/R3Y7...,Reviewed in India on 14 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
3,B09G9BL5CP,India,24-06-2024,True,5,Product,100% genuine,https://www.amazon.in/gp/customer-reviews/R1P9...,Reviewed in India on 24 June 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
4,B09G9BL5CP,India,18-05-2024,True,5,Good product,Happy to get the iPhone 13 in Amazon offer,https://www.amazon.in/gp/customer-reviews/R1XI...,Reviewed in India on 18 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   productAsin        299 non-null    object
 1   country            299 non-null    object
 2   date               299 non-null    object
 3   isVerified         299 non-null    bool  
 4   ratingScore        299 non-null    int64 
 5   reviewTitle        299 non-null    object
 6   reviewDescription  289 non-null    object
 7   reviewUrl          299 non-null    object
 8   reviewedIn         299 non-null    object
 9   variant            299 non-null    object
 10  variantAsin        299 non-null    object
dtypes: bool(1), int64(1), object(9)
memory usage: 23.8+ KB


None

In [8]:
import re
import string

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    else:
        text = "" # Handle non-string data by making it an empty string
    return text

df['cleaned_review'] = df['reviewDescription'].apply(preprocess_text)

display(df[['reviewDescription', 'cleaned_review']].head())

,reviewDescription,cleaned_review
0,"Every thing is good about iPhones, there's not...",every thing is good about iphones theres nothi...
1,"It look so fabulous, I am android user switche...",it look so fabulous i am android user switched...
2,I tried to flip camera while recording but no ...,i tried to flip camera while recording but no ...
3,100% genuine,100 genuine
4,Happy to get the iPhone 13 in Amazon offer,happy to get the iphone 13 in amazon offer


In [16]:
!pip install transformers

from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", truncation=True)

# Perform sentiment analysis on the cleaned reviews in batches
batch_size = 16 # You can adjust the batch size based on your available memory

sentiments = []
for i in range(0, len(df), batch_size):
    batch = df['cleaned_review'][i:i+batch_size].tolist()
    # Ensure all elements in the batch are strings and handle None values
    batch = [str(review) if review is not None else "" for review in batch]
    results = sentiment_analyzer(batch)
    sentiments.extend([r['label'] for r in results])

df['sentiment'] = sentiments

display(df[['reviewDescription', 'cleaned_review', 'sentiment']].head())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


KeyError: 'cleaned_review'

In [10]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.3 MB/s eta 0:00:00


In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    if isinstance(text, str):
        score = analyzer.polarity_scores(text)
        # Classify based on compound score
        if score['compound'] >= 0.05:
            return 'POSITIVE'
        elif score['compound'] <= -0.05:
            return 'NEGATIVE'
        else:
            return 'NEUTRAL'
    else:
        return None # Or 'NEUTRAL' depending on how you want to handle non-string data


df['sentiment'] = df['cleaned_review'].apply(vader_sentiment)

display(df[['reviewDescription', 'cleaned_review', 'sentiment']].head())

,reviewDescription,cleaned_review,sentiment
0,"Every thing is good about iPhones, there's not...",every thing is good about iphones theres nothi...,NEGATIVE
1,"It look so fabulous, I am android user switche...",it look so fabulous i am android user switched...,POSITIVE
2,I tried to flip camera while recording but no ...,i tried to flip camera while recording but no ...,NEGATIVE
3,100% genuine,100 genuine,NEUTRAL
4,Happy to get the iPhone 13 in Amazon offer,happy to get the iphone 13 in amazon offer,POSITIVE


In [19]:
!pip install transformers
!pip install sentencepiece # sentencepiece is a dependency for some transformers models

from transformers import pipeline
import re
import string

# Define the preprocess_text function again to ensure it's available in this cell
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    else:
        text = "" # Handle non-string data by making it an empty string
    return text

# Apply preprocessing just before summarization to ensure the column exists
df['cleaned_review'] = df['reviewDescription'].apply(preprocess_text)


# Load the summarization pipeline
# Using a smaller model for potentially faster inference
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", revision="a4f8f3e")

# Summarize the cleaned reviews in batches
batch_size = 8 # You can adjust the batch size based on your available memory and the model

summaries = []
for i in range(0, len(df), batch_size):
    batch = df['cleaned_review'][i:i+batch_size].tolist()
    # Ensure all elements in the batch are strings and handle None values
    batch = [str(review) if review is not None else "" for review in batch]
    # Adjust max_length and min_length based on expected summary length
    results = summarizer(batch, max_length=50, min_length=10, do_sample=False)
    summaries.extend([r['summary_text'] for r in results])

df['summary'] = summaries

# Display the original reviews, cleaned reviews, sentiment, and summaries for the sample
display(df[['reviewDescription', 'cleaned_review', 'sentiment', 'summary']].head())

Device set to use cpu
Your max_length is set to 50, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)
Your max_length is set to 50, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 50, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 50, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_le

KeyError: "['sentiment'] not in index"

In [22]:
!pip install streamlit matplotlib

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Ensure df is available from the global scope and store it in session state
# This assumes the preceding cells (loading, preprocessing, sentiment) have been run successfully
if 'df' in globals():
    st.session_state['df'] = df
else:
    st.error("DataFrame 'df' not found in global scope. Please run the data loading and analysis cells first.")
    st.stop() # Stop Streamlit execution if df is not found

# Use the DataFrame from session state
df_app = st.session_state['df']

st.title("Customer Review Sentiment Analyzer")

st.write("Analyze customer reviews and their sentiments.")

# Add a sentiment filter
# Check if 'sentiment' column exists before filtering
if 'sentiment' in df_app.columns:
    sentiment_filter = st.selectbox("Filter by Sentiment", ['All', 'POSITIVE', 'NEGATIVE', 'NEUTRAL'])

    filtered_df = df_app.copy()
    if sentiment_filter != 'All':
        filtered_df = df_app[df_app['sentiment'] == sentiment_filter]

    st.write(f"Displaying {len(filtered_df)} reviews with '{sentiment_filter}' sentiment.")

    # Display the filtered DataFrame
    # Check if 'cleaned_review' and 'sentiment' columns exist before displaying
    cols_to_display = ['reviewDescription']
    if 'cleaned_review' in filtered_df.columns:
        cols_to_display.append('cleaned_review')
    if 'sentiment' in filtered_df.columns:
        cols_to_display.append('sentiment')

    st.dataframe(filtered_df[cols_to_display].head(100))

    # Add a sentiment distribution chart
    st.write("## Sentiment Distribution")

    sentiment_counts = df_app['sentiment'].value_counts()

    fig, ax = plt.subplots()
    sentiment_counts.plot(kind='bar', ax=ax, color=['green', 'red', 'blue'])
    ax.set_title('Distribution of Sentiments')
    ax.set_xlabel('Sentiment')
    ax.set_ylabel('Number of Reviews')
    plt.xticks(rotation=0)
    st.pyplot(fig)

else:
    st.warning("Sentiment analysis has not been performed yet or the 'sentiment' column is missing. Please ensure the sentiment analysis cell was run successfully.")


# You can add more interactive elements here, like searching for keywords,
# or even adding the on-demand summarization feature later.

2025-09-03 21:19:53.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 21:19:53.563 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
!pip install gradio transformers sentencepiece

import gradio as gr
import pandas as pd
from transformers import pipeline
import re
import string

# Ensure df is available from the global scope
if 'df' not in globals():
    print("DataFrame 'df' not found in global scope. Please run the data loading and analysis cells first.")
    df = pd.DataFrame() # Create an empty DataFrame to avoid errors
else:
    # Use the existing df
    pass # df is already available

# Load the summarization pipeline
# Using a smaller model for potentially faster inference
# Initialize the summarizer here so it's loaded when the app starts
try:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", revision="a4f8f3e")
except Exception as e:
    print(f"Could not load summarization pipeline: {e}")
    summarizer = None # Set to None if loading fails


def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    else:
        text = "" # Handle non-string data by making it an empty string
    return text

# Apply preprocessing if not already done (e.g., if the notebook was run out of order)
if 'cleaned_review' not in df.columns:
    df['cleaned_review'] = df['reviewDescription'].apply(preprocess_text)

# Define the function to get filtered reviews
def get_reviews(sentiment_filter='All'):
    if 'sentiment' in df.columns:
        filtered_df = df.copy()
        if sentiment_filter != 'All':
            filtered_df = df[df['sentiment'] == sentiment_filter]

        # Select columns to display, checking for their existence
        cols_to_display = ['reviewDescription']
        if 'cleaned_review' in filtered_df.columns:
            cols_to_display.append('cleaned_review')
        if 'sentiment' in filtered_df.columns:
            cols_to_display.append('sentiment')
        # Add index for easy selection in summarization
        display_df = filtered_df[cols_to_display].reset_index()
        display_df = display_df.rename(columns={'index': 'Review Index'})
        return display_df.head(100)
    else:
        return pd.DataFrame({'Error': ["Sentiment analysis has not been performed yet or the 'sentiment' column is missing."]})

# Define the function to get a summary for a specific review index
def get_review_summary(review_index):
    if summarizer is None:
        return "Summarization model could not be loaded."
    try:
        # Get the original index from the DataFrame
        original_index = int(review_index)
        if original_index in df.index:
            review_text = df.loc[original_index, 'cleaned_review']
            if review_text:
                # Summarize the review
                summary = summarizer(review_text, max_length=50, min_length=10, do_sample=False)
                return summary[0]['summary_text']
            else:
                return "Review text is empty."
        else:
            return "Invalid Review Index."
    except ValueError:
        return "Please enter a valid integer Review Index."
    except Exception as e:
        return f"An error occurred during summarization: {e}"


# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("## Customer Review Sentiment Analyzer and Summarizer")
    gr.Markdown("Analyze customer reviews and their sentiments. Get summaries for individual reviews.")

    with gr.Row():
        with gr.Column():
            sentiment_dropdown = gr.Dropdown(['All', 'POSITIVE', 'NEGATIVE', 'NEUTRAL'], label="Filter by Sentiment", value='All')
            review_dataframe = gr.Dataframe(row_count=5, interactive=False)

            # Link the dropdown to update the dataframe
            sentiment_dropdown.change(fn=get_reviews, inputs=sentiment_dropdown, outputs=review_dataframe)

            gr.Markdown("### Get Summary for a Specific Review")
            review_index_input = gr.Number(label="Enter Review Index from the table above")
            summarize_button = gr.Button("Get Summary")
            review_summary_output = gr.Textbox(label="Review Summary")

            # Link the summarize button to the get_review_summary function
            summarize_button.click(fn=get_review_summary, inputs=review_index_input, outputs=review_summary_output)

    # Initial load of reviews
    iface.load(fn=get_reviews, inputs=sentiment_dropdown, outputs=review_dataframe)


iface.launch(share=True)

Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27d57d53e966282543.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
